In [2]:
%reload_ext autoreload
%autoreload 2

import plot_utils as pu
import utils as ut
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'monospace'

In [3]:
fold_data = '../data/espectra/'
# list files in fold_data
files = os.listdir(fold_data)
paths = [fold_data + f for f in files]

spectra = {}
x_max, x_min = 0, 1e5
for i, path in enumerate(paths):
    name = files[i].split('.')[0]
    spectra[name] = {}

    df = pd.read_csv(path, sep="\s+", header=None)
    x, y = df[0].values, df[1].values

    x_min_, x_max_ = np.min(x), np.max(x)

    if x_min_ < x_min:
        x_min = x_min_
    if x_max_ > x_max:
        x_max = x_max_

    y_min, y_max = np.min(y), np.max(y)
    dif = np.abs(y_max - y_min)

    spectra[name]['x'] = x
    spectra[name]['y'] = y
    spectra[name]['y_min'] = y_min
    spectra[name]['y_max'] = y_max
    spectra[name]['dif'] = dif

#spectra = ut.dict_swap(spectra, 1, -1)

spectra = {key: spectra[key] for key in ut.ordered_stellar_spectra}


# get last element in spectra
last = list(spectra.keys())[0]
spectra[last]['y']
# convert to nan values above 2
spectra[last]['y'][spectra[last]['y'] > 1.5] = np.nan

In [18]:
rat = 0.75
size = 1720
fig_w, fig_h = size*rat, size
subplots = (1, 1)
fig, axs, fs, gs = pu.initialize_figure(fig_w=fig_w, fig_h=fig_h, 
                                        ratio=None, theme='dark', 
                                        subplots=subplots, text_size=0.75, 
                                        color='#000000', grid=False)
lw, ts, ss = 0.1, 0.9, 0.7
ax = axs[0][0]

for i, key in enumerate(spectra.keys()):
    x, y = spectra[key]['x'], spectra[key]['y']
    y_min, y_max = spectra[key]['y_min'], spectra[key]['y_max']
    dif = spectra[key]['dif']

    y = y + i*1

    color = ut.color_spectra(key.split('_')[-1], cmap='turbo', range=[0.1,0.9])
    ax.plot(x, y, lw=lw*fs, label='Data', color=color)

    gap = 0.07*fs

    split = key.split('_')
    if len(split) == 3:
        star_name = split[0] + ' ' + split[1]
        spectral_type = split[2]
    else:
        star_name = key.split('_')[0]
        spectral_type = key.split('_')[1]
    ax.text(x_max, np.mean(y[-10:])+gap, star_name, fontsize=fs*ts*1, ha='right', va='center', color='k',
            c=color)
    ax.text(x_min-10, y[0], spectral_type, fontsize=fs*ts*1.2, ha='right', va='center', color='k',
            fontweight='bold', c=color)


# plotting Spectral Lines

spectral_lines = ut.spectral_lines

k = 0
for line in spectral_lines: 
    
    vals = spectral_lines[line]
    if isinstance(vals, int):
        vals = [vals]

    for val in vals:
        if val < x_max:

                m = np.mod(k,4)
                gap = -0.4

                ymi, yma = 0, 30.5

                ax.plot([val, val], [ymi, yma], ls='--', lw=lw*fs*0.5, color='w', alpha=0.5, zorder=-1)
                ax.text(val+2, yma+gap*m, line, fontsize=fs*ts*0.85,
                        ha='left', va='top', color='w', alpha=1,
                        fontname='Arial')
                k+=1


ax.set_xlim(x_min-150, x_max+50)

# ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel('Wavelength (\u00C5)', fontsize=1.3*fs*ts)
# give me the symbol for amstrong



savefold = '../figures/'
if not os.path.exists(savefold):
    # create the folder if it does not exist
    os.makedirs(savefold)
figname = 'example_spectra.png'
savepath = savefold + figname

fig.savefig(savepath, dpi=300, bbox_inches='tight')

plt.close()